Stock Sentiment Analysis of Finviz website using groq API

In [8]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


Extracting the data of amazon, google and meta stocks news from finviz website by web scrapping.
Financial Visualzaion (FinViz) is a website that brings together market data, financial news and analytics tools for investors and traders.

In [3]:
from urllib.request import urlopen, Request # Importing necessary modules for web scraping
from bs4 import BeautifulSoup # Importing BeautifulSoup for HTML parsing
import pandas as pd # Importing pandas for data manipulation

In [4]:
# URL of the Finviz website where stock news is located
finviz_url = 'https://finviz.com/quote.ashx?t='

# List of stock tickers for which we want to fetch news
tickers = ['AMZN', 'GOOG', 'META']

In [9]:
news_tables = {}
# Loop through each ticker, fetch the corresponding news webpage, parse its HTML content
for ticker in tickers:
    url = finviz_url + ticker

    req = Request(url=url, headers={'user-agent': '_my-app_'})
    response = urlopen(req)

    html = BeautifulSoup(response, features='html.parser')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table


parsed_data = []
# Extracting news tables, and storing them in the news_tables dictionary.
for ticker, news_table in news_tables.items():
    for row in news_table.findAll('tr'):
        if row.a and row.td:
            title = row.a.text
            date_data = row.td.text.strip().split(' ')

            if len(date_data) == 1:
                date = pd.to_datetime('today').strftime('%Y-%m-%d')
                time = date_data[0]
            else:
                date = date_data[0]
                time = date_data[1]

            parsed_data.append([ticker, date, time, title])

# Collect the parsed data and store in the dataframe
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])

In [6]:
# Checkout the first five rows of the dataframe
df.head()

,ticker,date,time,title
0,AMZN,Today,08:00AM,Inside Amazons race to become a $10 trillion g...
1,AMZN,2024-06-21,07:46AM,Amazon will stop using those little plastic pi...
2,AMZN,2024-06-21,07:22AM,3 Penny Stocks to Buy Now: June 2024
3,AMZN,2024-06-21,07:00AM,Microsoft's AI Edge: Why MSFT Stock Is Poised ...
4,AMZN,2024-06-21,06:52AM,Hype-nosis: Snap Out of the Trance and See The...


**Groq:**

Groq is a company that provides a hardware platform for efficient AI inference.

They’ve developed the world’s first Language Processing Unit™, or LPU. The Groq

LPU is designed to accelerate AI workloads, including inference for large

language models.

Here I used the groq api to analyse the sentiments of stock news article

In [22]:
from groq import Groq

# Enter your api key
client = Groq(api_key="Enter_your_api_here")

# Function to perform sentiment analysis using Groq API
def sentiment_analysis(text):
    chat_completion = client.chat.completions.create(
        messages= [ {"role": "system", "content": """You are trained to analyze and detect the sentiment of given text.
                                                          If you're unsure of an answer, you can say "not sure" and recommend users to review manually."""},
                    {"role": "user", "content": f"""Identify the sentiment towards the stocks from the news article , where the sentiment score should be from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral.
                              Article : {text}"""}],
        model="llama3-8b-8192", # specify model used to analyse the sentments
        temperature=0.5,
        max_tokens=1024,
        stop=None,
        stream=False,
    )

    response_text = chat_completion.choices[0].message.content.strip().lower()
    return response_text

The sentiment score is divided as:

-10 for most negative

+10 for most positive

0 for neutral

Analysing the sentiments of different news articles with explanation:

In [23]:
print(sentiment_analysis(df['title'][13]))

after analyzing the article, i would rate the sentiment towards the stocks as +6.

the article appears to be positive towards the companies mentioned, particularly mag 7, which is described as laying down "ai 'bedrock'", implying a strong foundation for their ai technology. the phrase "as nvidia, apple show cracks" is likely referring to the challenges faced by those companies, but it's not the main focus of the article, which is on mag 7's achievement.

overall, the tone is optimistic and highlights mag 7's success, which suggests a positive sentiment towards the company.


In [43]:
print(sentiment_analysis(df['title'][20]))

after analyzing the article, i would say that the sentiment towards the stocks is:

sentiment score: +8

reasoning:

the article is discussing how amazon is disrupting the streaming advertising market, which is a positive development for amazon's stock. the tone of the article is informative and neutral, but the overall implication is that amazon's entry into the market is a significant event that could have a positive impact on its stock price. there is no mention of negative aspects or concerns about amazon's stock, which suggests a positive sentiment.


In [45]:
print(sentiment_analysis(df['title'][75]))

the sentiment towards the stocks in this news article is likely to be negative. the article is reporting on a fine imposed on amazon by the state of california, which implies that amazon is being penalized for violating labor laws. this could potentially have a negative impact on the company's stock price.

sentiment score: -5

note: the sentiment score is not extremely negative (-10), as the fine is a specific issue and may not have a significant impact on the company's overall performance. however, it is still negative due to the implication that amazon is being penalized for violating labor laws.


In [48]:
print(sentiment_analysis(df['title'][30]))

after analyzing the article, i would rate the sentiment towards the stocks as 8 out of 10, leaning towards being positive.

the article is discussing the potential for reddit to continue growing, which suggests a positive outlook for the company's stock performance. the use of phrases such as "more room to grow" and "analyst" implies a sense of optimism and confidence in reddit's future prospects.

here's a breakdown of the sentiment:

* positive words/phrases: "more room to grow", "analyst"
* neutral words/phrases: none
* negative words/phrases: none

overall, the sentiment is strongly positive, indicating a high likelihood of the stock performing well in the future.
